In [1]:
%reload_ext autoreload
%autoreload all

import tools

### Tool for listing data sources

In [2]:
tools.list_data_sources()

[{'name': 'employees',
  'description': 'Employees Database',
  'type': 'SQLite',
  'file': 'employees.sqlite'}]

### Tool for describing a data source

In [3]:
print(tools.describe_data_source('employees')[:500] + '\n...')

# Employee Database

This SQLite database contains employee information for a company with the following structure:

## Tables

### employee
**Primary table containing employee personal information**
- `emp_no` (INTEGER, PRIMARY KEY): Unique employee number
- `birth_date` (DATE): Date of birth
- `first_name` (TEXT): Employee's first name
- `last_name` (TEXT): Employee's last name
- `gender` (TEXT): Gender ('M' or 'F')
- `hire_date` (DATE): Date employee was hired

### department
**Department inf
...


### Tool for getting the database schema for SQLite data sources

In [4]:
print(tools.sqlite_get_schema('employees')[:500] + '\n...')

-- Tables
CREATE TABLE department (
    dept_no     TEXT            NOT NULL,
    dept_name   TEXT            NOT NULL,
    PRIMARY KEY (dept_no),
    UNIQUE      (dept_name)
);
CREATE TABLE dept_emp (
    emp_no      INTEGER         NOT NULL,
    dept_no     TEXT            NOT NULL,
    from_date   DATE            NOT NULL,
    to_date     DATE            NOT NULL,
    FOREIGN KEY (emp_no)  REFERENCES employee (emp_no)   ON DELETE CASCADE,
    FOREIGN KEY (dept_no) REFERENCES department (dept_
...


### Tool for querying a SQLite data source

In [5]:
print(tools.sqlite_query('employees', 'SELECT * from employee')[:500] + '\n...')

emp_no | birth_date | first_name | last_name | gender | hire_date
-----------------------------------------------------------------
10001 | 1953-09-02 | Georgi | Facello | M | 1986-06-26
10002 | 1964-06-02 | Bezalel | Simmel | F | 1985-11-21
10003 | 1959-12-03 | Parto | Bamford | M | 1986-08-28
10004 | 1954-05-01 | Chirstian | Koblick | M | 1986-12-01
10005 | 1955-01-21 | Kyoichi | Maliniak | M | 1989-09-12
10006 | 1953-04-20 | Anneke | Preusig | F | 1989-06-02
10007 | 1957-05-23 | Tzvetan | Zie
...


### Setting up an agent

In [6]:
from smolagents import OpenAIServerModel, CodeAgent, ActionStep

model = OpenAIServerModel(model_id="gpt-4.1-mini")

agent = CodeAgent(
    model=model,
    tools=[tools.list_data_sources, tools.describe_data_source, tools.sqlite_get_schema, tools.sqlite_query],
)

def count_action_steps(agent):
    """Count the number of action steps in an agent's memory."""
    return sum(1 for step in agent.memory.steps if isinstance(step, ActionStep))

### Running the agent with an example question

In [7]:
final_answer = agent.run("Which department has the highest number of employees?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which department has the highest number of employees?                                                           │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4.1-mini ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  data_sources = list_data_sources()                                                                               
  print(data_sources)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[{'name': 'employees', 'description': 'Employees Database', 'type': 'SQLite', 'file': 'employees.sqlite'}]

Out: None

[Step 1: Duration 4.31 seconds| Input tokens: 2,103 | Output tokens: 85]

...

[Step 3: Duration 4.19 seconds| Input tokens: 7,414 | Output tokens: 332]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Development")                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Development

[Step 4: Duration 1.63 seconds| Input tokens: 10,769 | Output tokens: 376]

### Running questions from evals.py

In [8]:
from smolagents.monitoring import LogLevel
from tqdm.auto import tqdm
from reward import compute_reward
from evals import evals_dataset

silent_agent = CodeAgent(
    model=model,
    tools=[tools.list_data_sources, tools.describe_data_source, tools.sqlite_get_schema, tools.sqlite_query],
    verbosity_level=LogLevel.ERROR
)

for item in tqdm(evals_dataset):

    final_answer = silent_agent.run(item['question'])
    n_actions = count_action_steps(agent)
    reward = compute_reward(final_answer, item['answer'], n_actions, item['optimal_actions'])

    print(f"Question: {item['question']}")
    print(f"Agent's answer: {final_answer}")
    print(f"Expected answer: {item['answer']}")
    print(f"Number of performed action steps: {n_actions}")
    print(f"Number of optimal action steps: {item['optimal_actions']}")
    print(f"Reward: {reward:.3f}\n")

  0%|          | 0/2 [00:00<?, ?it/s]

Question: Who is the most paid employee? Return only the name of the emploeyee.
Agent's answer: Arno Kumaresan
Expected answer: Arno Kumaresan
Number of performed action steps: 4
Number of optimal action steps: 4
Reward: 1.000

Question: Which department has the highest number of employees?
Agent's answer: Development
Expected answer: Development
Number of performed action steps: 4
Number of optimal action steps: 4
Reward: 1.000

